## 769. 最多能完成排序的块

最简单的理解：假如数组前n个元素的最大值等于n-1，因为数组的元素不重复，且为非负整数，因此只能包含元素0到n-1。那么可以将其切块出来排序。
因此，只需要遍历数组，当最大值等于index-1时进行切块即可。

In [16]:
# beat 90%
def maxChunksToSorted(arr):
    """
    :type arr: List[int]
    :rtype: int
    """
    max_num = 0
    count = 0
    for i in range(len(arr)):
        max_num = max(arr[i],max_num)
        if i == max_num:
            count += 1
    return count
maxChunksToSorted([0,1,2,3,6,5,4])

5

## 775. 全局倒置与局部倒置
留意几点：
- 局部倒置一定属于全局倒置
- 假如存在全局倒置但非局部倒置，就需要返回False。
- 考虑一个最简单的情况：[0,1,2]。
    * 对于仅对调相邻元素的情况（即[1,0,2]，[0,2,1]），满足全局倒置和局部倒置数量一直，返回True。
    * 对于对调间隔超过1的元素（即[2,1,0]），全局倒置数大于局部倒置。
- 再次留意，传入数组A是一个0到n-1的一种排列的数组。我们知道，对于数组[0,1,2,...,n-1]如果对调间隔超过1的元素，就会出现全局倒置数大于局部倒置。所以，只要判断A中哪些元素进行了跨度超过1的对调就行。数组I=[0,1,2,...,n-1]满足条件I[i]=i，假如进行跨度超过1的对调，必然出现一种情况：abs(I[i]-i)>1,到这里我们成功找到了判断条件。

In [29]:
# beat 67%
def isIdealPermutation(A):
    """
    :type A: List[int]
    :rtype: bool
    """

    for i in range(len(A)):
        if abs(A[i] - i)>=2:
            return False
    return True
isIdealPermutation([1,2,0])

False

## 792. 匹配子序列的单词数

In [49]:
# beat 37%
def numMatchingSubseq(S, words):
    """
    :type S: str
    :type words: List[str]
    :rtype: int
    """
    dic_words = {} #记录words的字典形式，以每个word的首字母作为key值
    
    for word in words:
        if word[0] in dic_words:
            dic_words[word[0]].append(word)
        else:
            dic_words[word[0]] = [word]
    count = 0
    for ch in S:
        if ch in dic_words:
            for i in range(len(dic_words[ch])-1,-1,-1):
                ch_word = dic_words[ch].pop(i)
                if len(ch_word) == 1:
                    count += 1
                else:
                    if ch_word[1] in dic_words:
                        dic_words[ch_word[1]].append(ch_word[1:])
                    else:
                        dic_words[ch_word[1]] = [ch_word[1:]]
    return count
numMatchingSubseq("kjalibjklaib",["a", "bb", "acd", "ace"])

2

## 795. 区间子数组个数

因为要求子数组最大值必须处于[L,R]之间，因此，遍历数组，截取出最大值在[L,R]之间的最长子数组。截取方法其实就是拿数组中大于R的值作为隔断。
得到最长子数组后，需要计算符合条件的子数组数量，计算方法为计算所有子数组数量，减去不符合的子数组数量。

In [5]:
# beat 16%
def numSubarrayBoundedMax(A, L, R):
    """
    :type A: List[int]
    :type L: int
    :type R: int
    :rtype: int
    """
    
    count = 0
    continious_count = 0
    count_2 = 0
    continious_count_2 = 0
    max_num = float('-inf')
    for ele in A:
        max_num = max(max_num,ele)
        if ele < L:
            continious_count += 1
            continious_count_2 += 1
        elif ele <= R:
            continious_count += 1
            count_2 -= (continious_count_2+1)*continious_count_2/2
            continious_count_2 = 0
        else:
            count += (continious_count+1)*continious_count/2
            max_num = float('-inf')
            if continious_count > 0:
                count_2 -= (continious_count_2+1)*continious_count_2/2
                count += count_2
            count_2 = 0
            continious_count_2 = 0
            continious_count = 0
        
    if continious_count > 0:
        count += (continious_count+1)*continious_count/2
        count_2 -= (continious_count_2+1)*continious_count_2/2
        count += count_2
    return int(count)
numSubarrayBoundedMax([876,880,482,260,132,421,732,703,795,420,871,445,400,291,358,589,617,202,755,810,\
                       227,813,549,791,418,528,835,401,526,584,873,662,13,314,988,101,299,816,833,224,160,\
                       852,179,769,646,558,661,808,651,982,878,918,406,551,467,87,139,387,16,531,307,389,939,\
                       551,613,36,528,460,404,314,66,111,458,531,944,461,951,419,82,896,467,353,704,905,705,760,\
                       61,422,395,298,127,516,153,299,801,341,668,598,98,241],658,719)

19

## 825. 适龄的朋友

题目中，满足以下3个条件的任一个都不能发好友请求。

    age[B] <= 0.5 * age[A] + 7
    age[B] > age[A]
    age[B] > 100 && age[A] < 100

细看一下，可以发现条件2已包含了条件3。所以仅考虑条件1、2就行。再看下，发现当age[A]<=14时，条件1、2一定有一个会满足，意味着这有14岁以上的人才能发好友请求。

- 方案一
    * 采用暴力解法，遍历。提交后超时。
- 方案二
    * 改进点1：原来需要满足条件0.5 * age[A] + 7 < age[B] <= age[A]，将ages进行倒序排序后，只需要比较age[A]往后的元素，且判断条件简化为0.5 * age[A] + 7 < age[B]。
    * 改进点2：我们发现，假如age[B]满足0.5 * age[A] + 7 < age[B]，那么，age[B]往前到age[A]的元素都满足，那么就可以跳出循环，减少遍历计算量。
    * 改进点3：age[A]满足0.5 * age[A] + 7 < age[B]，那么，当age[A]变大时，age[B]只要在满足上一个age[A]的age[B]基础上往左遍历就行，无需再从ages数组的尾部开始遍历。
- 方案三
    * 因为影响能否发送好友请求的因素只有年龄，所以统计数组中各个年龄的人数，针对年龄进行遍历。计算量从方案二P（人数）的量级变为N（年龄）的量级，大大减少计算量。

In [ ]:
# can't pass
# 方案一
def numFriendRequests(ages):
    """
    :type ages: List[int]
    :rtype: int
    """

    ages.sort(reverse=True)
    ages.insert(0,-1)
    counted = [0.5*ele+7 for ele in ages]
    count = 0
    repeat_nums = 0
    for i in range(1,len(ages)):
        ele = ages[i]
        if ele == ages[i-1]:
            repeat_nums += 1
        else:
            repeat_nums = 0
        if ele <= 14:
            break
        for j in range(i+1-repeat_nums,len(ages)):
            if counted[i] < ages[j] <= ele:
                count += 1
    return count

In [30]:
# beat 29%
# 方案二
def numFriendRequests(ages):
    """
    :type ages: List[int]
    :rtype: int
    """

    ages.sort(reverse=True)
    ages.append(121)
    counted = [0.5*ele+7 for ele in ages]
    count = 0
    repeat_nums = 0
    j = len(ages)-2
    for i in range(len(ages)-2,-1,-1):
        ele = ages[i]
        if ele <= 14:
            continue
        if ele == ages[i+1]:
            repeat_nums += 1
        else:
            repeat_nums = 0
        while j >= i+1:
            if counted[i] < ages[j]:
                count += j - i
                break
            j -= 1
        count += repeat_nums
    return count
numFriendRequests([108,115,5,24,82])

3

In [ ]:
# beat 70%
# 方案三
def numFriendRequests(ages):
    """
    :type ages: List[int]
    :rtype: int
    """
    cnt = [0] * 121
    for age in ages:
        cnt[age] += 1

    ans = 0
    for i in range(15, 121):
        if cnt[i] == 0: continue
        for j in range(math.floor(i*0.5+7)+1, i):
            ans += cnt[i] * cnt[j]
        ans += (cnt[i])*(cnt[i]-1)
    return ans

## 152. 乘积最大子序列

- 方案一
    * 暴力解法，运行时间超时
- 方案二
    * 输入序列元素为整数，因此，将序列以0元素进行切分。得到非零元素子序列。这些子序列的元素绝对值大于等于1。
    * 对于每一个子序列，如果负元素数量为偶数，那么子序列所有元素相乘即为该子序列的最大子序列乘积。如负元素数量为奇数，只需要考虑去掉第一个负元素之前的序列的乘积，以及去掉最后一个负元素之后的序列的乘积，两者的最大值即为该子序列的最大乘积。
    * 举个例子：[2,-1,3,4,2,-2,2,0,2]。0元素切分为两个子序列：[2,-1,3,4,2,-2,2],[2]。对于第一个，负数个数为2个，偶数，因此第一个子序列的最大乘积为2*(-1)*3*4*2*(-2)*2=192。第二个子序列最大乘积为2。同时考虑0元素，则该数组最大子序列乘积为max(192,2,0)=192
    * 再举个例子：[2,-1,3,-1,4,2,-2,2,0,2]。0元素切分为两个子序列：[2,-1,3,-1,4,2,-2,2],[2]。对于第一个，负数个数为3个，奇数，因此，去掉第一个负元素-1之前的元素，子序列为[3,-1,4,2,-2,2],乘积为96；去掉最后一个负元素-2之后的元素，子序列为[2,-1,3,-1,4,2]，乘积为48，则第一个子序列的最大乘积为max(96,48)=96。第二个子序列最大乘积为2。同时考虑0元素，则该数组最大子序列乘积为max(96,2,0)=96

In [37]:
# can't pass
# 方案一
def maxProduct(nums):
    """
    :type nums: List[int]
    :rtype: int
    """
    n = len(nums)
    product = 1
    max_product = float('-inf')
    for i in range(n):
        product = nums[i]
        max_product = max(max_product, product)
        for j in range(i+1,len(nums)):
            product *= nums[j]
            max_product = max(max_product, product)
    return max_product
maxProduct([-4,-3,-2])

12

In [43]:
# beat 93%
# 方案二
def maxProduct(nums):
    """
    :type nums: List[int]
    :rtype: int
    """
    max_product = float('-inf')  #记录最大乘积
    sub_product = 1  #记录被0元素分割后的子序列的乘积
    first_neg = 1  #记录第一个负元素及之前的元素的乘积
    last_neg = 1  #记录最后一个负元素及之后的元素的乘积
    neg_count = 0 #记录子序列负数的数量
    pos_count = 0 #记录子序列正数的数量
    for ele in nums:
        if ele > 0:
            pos_count += 1
            sub_product *= ele
            last_neg *= ele
        elif ele < 0:
            neg_count += 1
            sub_product *= ele
            if first_neg == 1:
                first_neg = sub_product
            last_neg = ele
        else:
            if neg_count % 2 == 0:
                if pos_count + neg_count:
                    max_product = max(max_product,sub_product)
                else:
                    max_product = max(max_product,0)
            else:
                if pos_count+neg_count > 1:
                    max_product = max(max_product,sub_product/first_neg, sub_product/last_neg,0)
                else:
                    max_product = max(max_product,sub_product,0)
            sub_product = 1
            first_neg = 1
            last_neg = 1
            neg_count = 0
            pos_count = 0
    if neg_count % 2 == 0:
        if pos_count + neg_count:
            max_product = max(max_product,sub_product)
        else:
            max_product = max(max_product,0)
    else:
        if pos_count+neg_count > 1:
            max_product = max(max_product,sub_product/first_neg, sub_product/last_neg)
        else:
            max_product = max(max_product,sub_product)
    return int(max_product)
maxProduct([0])

0

## 53. 最大子序和

动态规划法解决，时间复杂度$O(N)$。
分治方法时间复杂度$O(NlogN)$，不考虑。

In [41]:
# beat 84%
def maxSubArray(nums):
    """
    :type nums: List[int]
    :rtype: int
    """
    sum_1 = 0
    max_sum = float('-inf')
    for i in range(len(nums)):
        sum_1 += nums[i]
        max_sum = max(max_sum, sum_1)
        sum_1 = max(sum_1,0)
    return max_sum
maxSubArray([4,-3,4,-1,2,1,-5,4])

7

## 153. 寻找旋转排序数组中的最小值

In [65]:
# beat 88.8%
def findMin(nums):
    """
    :type nums: List[int]
    :rtype: int
    """
    def dfs(left,right):
        if right - left <=1:
            return min(nums[left],nums[right])
        mid = (left+right)//2
        if nums[left] > nums[mid] and nums[right] > nums[mid]:
            return dfs(left, mid)
        elif nums[left] < nums[mid] and nums[right] < nums[mid]:
            return dfs(mid+1, right)
        else:
            return nums[left]
    return dfs(0,len(nums)-1)
findMin([4,5,6,7,0,1,2])

0

## 162. 寻找峰值

In [73]:
# beat 76.4%
def findPeakElement(nums):
    """
    :type nums: List[int]
    :rtype: int
    """
    if len(nums)<=2:
        return nums.index(max(nums))
    for i in range(1,len(nums)-1):
        if nums[i]>nums[i-1] and nums[i]>nums[i+1]:
            return i
findPeakElement([1,2,1,3,5,6,4])

1

## 835. 图像重叠

留意该题的矩阵移动后，移出矩阵范围的元素是丢弃操作的。以下解法一为移出后不丢弃的方案（有点像给图片做symmetric的padding），解法二为移出后丢弃的方案。

In [96]:
# can't pass
# 解法一，移出后不丢弃
def largestOverlap(A, B):
    """
    :type A: List[List[int]]
    :type B: List[List[int]]
    :rtype: int
    """
    max_sum = 0
    n = len(A)
    for i in range(n):
        for j in range(n):
            sum_ = 0
            for k in range(n):
                for l in range(n):
                    sum_ += A[(k+i)%n][(l+j)%n] * B[k][l]
            max_sum = max(max_sum, sum_)
    return max_sum
largestOverlap([[1,0],[1,0]], [[0,1],[1,0]])

1

In [100]:
# beat 14
# 解法二，移出后丢弃。来自leetcode官方解答
def largestOverlap(A, B):
    """
    :type A: List[List[int]]
    :type B: List[List[int]]
    :rtype: int
    """
    import collections
    N = len(A)
    count = collections.Counter()
    for i, row in enumerate(A):
        for j, v in enumerate(row):
            if v:
                for i2, row2 in enumerate(B):
                    for j2, v2 in enumerate(row2):
                        if v2:
                            count[i-i2, j-j2] += 1
    print(count)
    return max(count.values() or [0])
largestOverlap([[0,1],[1,1]], [[1,1],[1,0]])

Counter({(0, 1): 2, (0, 0): 2, (1, 0): 2, (-1, 1): 1, (1, -1): 1, (1, 1): 1})


2

In [ ]:
aaa = '0e000000010000084001089f'
asc = []
for i in range(0,len(aaa),2):
    temp = aaa[i:i+2]
    temp2 = int(temp,16)
    asc.append(temp2)
print(asc)
asc = bytearray(asc)
crc16 = crcmod.predefined.Crc('modbus')
crc16.update(asc)
hex(crc16.crcValue)